In [1]:
import random as rd
import re
import math
import string

In [2]:
path = 'dict.txt'

In [3]:
# import ast
  
# # reading the data from the file
# with open(path) as f:
#     data = f.read()
  
# print("Data type before reconstruction : ", type(data))
      
# d = ast.literal_eval(data)
  
# print("Data type after reconstruction : ", type(d))
# print(d)

In [4]:
f = open(path, "r", encoding = 'utf-8')
# print(f.read())
for i in f:
    print("----")
    print(str(i))
    print('\n\n\n')

----
WorldCup:['1597017362628513793|2022-11-27T23:59:45.000Z|Edgar #WorldCup Driscoll #世界杯 Quenna #偏门 Iver #卡塔尔世界杯  https://t.co/sc6VXaDK1Z', '1594778590079770624|2022-11-27T23:58:54.000Z|@KSI When most of Americans recognized it as Soccer.. #WorldCup', '1597017132826599424|2022-11-27T23:58:50.000Z|RT @ManuAkanji: Confident and ready for Brasil test! #WorldCup #Swiss🇨🇭 https://t.co/pn6QPpDaIh', "1595426350998982663|2022-11-27T23:58:25.000Z|@Control2XY MEXC Twitter Space #WorldCup2022 Special - Episode 1 is here  Let's talk about the #WorldCup we watched in the past years! Set a reminder  https://t.co/x3JiMq6m5G… Join the Vote &amp; Earn event  https://t.co/cB1yIhy7aL", '1597016917029502976|2022-11-27T23:57:58.000Z|RT @KhaledBeydoun: The most Arab/African/Muslim photo from the #WorldCup https://t.co/rROIa2Uz7Q', '1597016906124656640|2022-11-27T23:57:56.000Z|RT @ManUtd: Representing United in the first week of the #WorldCup 😍\n\n#MUFC', '1597016874348576768|2022-11-27T23:57:48.000Z|RT @K

In [5]:
def pre_process_tweets(url):

    f = open(url, "r", encoding="utf8")
    tweets = list(f)
    print(tweets[:5])

    list_of_tweets = []

    for i in range(len(tweets)):

        # remove \n from the end after every sentence
        tweets[i] = tweets[i].strip('\n')

        # Remove the tweet id and timestamp
        tweets[i] = tweets[i][50:]

        # Remove any word that starts with the symbol @
        tweets[i] = " ".join(filter(lambda x: x[0] != '@', tweets[i].split()))

        # Remove any URL
        tweets[i] = re.sub(r"http\S+", "", tweets[i])
        tweets[i] = re.sub(r"www\S+", "", tweets[i])

        # remove colons from the end of the sentences (if any) after removing url
        tweets[i] = tweets[i].strip()
        tweet_len = len(tweets[i])
        if tweet_len > 0:
            if tweets[i][len(tweets[i]) - 1] == ':':
                tweets[i] = tweets[i][:len(tweets[i]) - 1]

        # Remove any hash-tags symbols
        tweets[i] = tweets[i].replace('#', '')

        # Convert every word to lowercase
        tweets[i] = tweets[i].lower()

        # remove punctuations
        tweets[i] = tweets[i].translate(str.maketrans('', '', string.punctuation))

        # trim extra spaces
        tweets[i] = " ".join(tweets[i].split())

        # convert each tweet from string type to as list<string> using " " as a delimiter
        list_of_tweets.append(tweets[i].split(' '))

    f.close()

    return list_of_tweets

In [6]:
def k_means(tweets, k=4, max_iterations=50):

    centroids = []

    # initialization, assign random tweets as centroids
    count = 0
    hash_map = dict()
    while count < k:
        random_tweet_idx = rd.randint(0, len(tweets) - 1)
        if random_tweet_idx not in hash_map:
            count += 1
            hash_map[random_tweet_idx] = True
            centroids.append(tweets[random_tweet_idx])

    iter_count = 0
    prev_centroids = []

    # run the iterations until not converged or until the max iteration in not reached
    while (is_converged(prev_centroids, centroids)) == False and (iter_count < max_iterations):

        print("running iteration " + str(iter_count))

        # assignment, assign tweets to the closest centroids
        clusters = assign_cluster(tweets, centroids)

        # to check if k-means converges, keep track of prev_centroids
        prev_centroids = centroids

        # update, update centroid based on clusters formed
        centroids = update_centroids(clusters)
        iter_count = iter_count + 1

    if (iter_count == max_iterations):
        print("max iterations reached, K means not converged")
    else:
        print("converged")

    sse = compute_SSE(clusters)

    return clusters, sse, centroids


In [7]:
def is_converged(prev_centroid, new_centroids):

    # false if lengths are not equal
    if len(prev_centroid) != len(new_centroids):
        return False

    # iterate over each entry of clusters and check if they are same
    for c in range(len(new_centroids)):
        if " ".join(new_centroids[c]) != " ".join(prev_centroid[c]):
            return False

    return True


In [8]:
def assign_cluster(tweets, centroids):

    clusters = dict()

    # for every tweet iterate each centroid and assign closest centroid to a it
    for t in range(len(tweets)):
        min_dis = math.inf
        cluster_idx = -1;
        for c in range(len(centroids)):
            dis = getDistance(centroids[c], tweets[t])
            # look for a closest centroid for a tweet

            if centroids[c] == tweets[t]:
                # print("tweet and centroid are equal with c: " + str(c) + ", t" + str(t))
                cluster_idx = c
                min_dis = 0
                break

            if dis < min_dis:
                cluster_idx = c
                min_dis = dis

        # randomise the centroid assignment to a tweet if nothing is common
        if min_dis == 1:
            cluster_idx = rd.randint(0, len(centroids) - 1)

        # assign the closest centroid to a tweet
        clusters.setdefault(cluster_idx, []).append([tweets[t]])
        # print("tweet t: " + str(t) + " is assigned to cluster c: " + str(cluster_idx))
        # add the tweet distance from its closest centroid to compute sse in the end
        last_tweet_idx = len(clusters.setdefault(cluster_idx, [])) - 1
        clusters.setdefault(cluster_idx, [])[last_tweet_idx].append(min_dis)

    return clusters

In [9]:
def update_centroids(clusters):

    centroids = []

    # iterate each cluster and check for a tweet with closest distance sum with all other tweets in the same cluster
    # select that tweet as the centroid for the cluster
    for c in range(len(clusters)):
        min_dis_sum = math.inf
        centroid_idx = -1

        # to avoid redundant calculations
        min_dis_dp = []

        for t1 in range(len(clusters[c])):
            min_dis_dp.append([])
            dis_sum = 0
            # get distances sum for every of tweet t1 with every tweet t2 in a same cluster
            for t2 in range(len(clusters[c])):
                if t1 != t2:
                    if t2 < t1:
                        dis = min_dis_dp[t2][t1]
                    else:
                        dis = getDistance(clusters[c][t1][0], clusters[c][t2][0])

                    min_dis_dp[t1].append(dis)
                    dis_sum += dis
                else:
                    min_dis_dp[t1].append(0)

            # select the tweet with the minimum distances sum as the centroid for the cluster
            if dis_sum < min_dis_sum:
                min_dis_sum = dis_sum
                centroid_idx = t1

        # append the selected tweet to the centroid list
        centroids.append(clusters[c][centroid_idx][0])

    return centroids


In [10]:
def getDistance(tweet1, tweet2):

    # get the intersection
    intersection = set(tweet1).intersection(tweet2)

    # get the union
    union = set().union(tweet1, tweet2)

    # return the jaccard distance
    return 1 - (len(intersection) / len(union))


In [11]:
def compute_SSE(clusters):

    sse = 0
    # iterate every cluster 'c', compute SSE as the sum of square of distances of the tweet from it's centroid
    for c in range(len(clusters)):
        for t in range(len(clusters[c])):
            sse = sse + (clusters[c][t][1] * clusters[c][t][1])

    return sse

In [12]:
if __name__ == '__main__':

    data_url = 'dict.txt'

    tweets = pre_process_tweets(data_url)
    print(tweets[:5])

    # default number of experiments to be performed
    experiments = 5

    # default value of K for K-means
    k = 3

    # for every experiment 'e', run K-means
    min_sse, req_k, req_cluster = math.inf, math.inf, []
    for e in range(experiments):

        print("------ Running K means for experiment no. " + str((e + 1)) + " for k = " + str(k))

        clusters, sse, centroids = k_means(tweets, k)

        # for every cluster 'c', print size of each cluster
        for c in range(len(clusters)):
            print(str(c+1) + ": ", str(len(clusters[c])) + " tweets")
            # # to print tweets in a cluster
            # for t in range(len(clusters[c])):
            #     print("t" + str(t) + ", " + (" ".join(clusters[c][t][0])))

        print("--> SSE : " + str(math.sqrt(sse)))
        if min_sse > sse:
            min_sse = sse
            req_k = k
            req_cluster = clusters
#         print('\n')
        
        print(centroids)


        # increment k after every experiment
        k = k + 1
#         print('----------')
#         print(type(clusters))
#         print(clusters.keys())
#         print((max(len(clusters.values()))))
#         print(len(max(clusters.items(), key=lambda x:len(x[1]))[1]), max(clusters.items(), key=lambda x:len(x[1]))[0])
        i = max(clusters.items(), key=lambda x:len(x[1]))[0]
        print(centroids[i])
        print('----------')
        print('\n')
        
        

['WorldCup:[\'1597017362628513793|2022-11-27T23:59:45.000Z|Edgar #WorldCup Driscoll #世界杯 Quenna #偏门 Iver #卡塔尔世界杯  https://t.co/sc6VXaDK1Z\', \'1594778590079770624|2022-11-27T23:58:54.000Z|@KSI When most of Americans recognized it as Soccer.. #WorldCup\', \'1597017132826599424|2022-11-27T23:58:50.000Z|RT @ManuAkanji: Confident and ready for Brasil test! #WorldCup #Swiss🇨🇭 https://t.co/pn6QPpDaIh\', "1595426350998982663|2022-11-27T23:58:25.000Z|@Control2XY MEXC Twitter Space #WorldCup2022 Special - Episode 1 is here  Let\'s talk about the #WorldCup we watched in the past years! Set a reminder  https://t.co/x3JiMq6m5G… Join the Vote &amp; Earn event  https://t.co/cB1yIhy7aL", \'1597016917029502976|2022-11-27T23:57:58.000Z|RT @KhaledBeydoun: The most Arab/African/Muslim photo from the #WorldCup https://t.co/rROIa2Uz7Q\', \'1597016906124656640|2022-11-27T23:57:56.000Z|RT @ManUtd: Representing United in the first week of the #WorldCup 😍\\n\\n#MUFC\', \'1597016874348576768|2022-11-27T23:57:48

running iteration 1
converged
1:  91 tweets
2:  55 tweets
3:  46 tweets
--> SSE : 11.70859633776143
[['000zlunagensoffcial', 'check', 'out', 'worldcup', 'fifa', 'ronaldo', 'bsc', 'altcoin', '100xgem', 'crypto', 'bscgemalert', 'worldcupsanta', 'fifaworldcup', 'fifaworldcup2022', 'nfts', 'nftgame', 'bscgems', 'crypto', 'nnworldcupsantann'], ['brazil', 'vs', 'switzerland', 'tickets', 'availableworldcup', 'worldcup', 'worldcupbrazil', 'brazil', 'football', 'sccoer', 'worldcupfever', 'france', 'topliketags', 'germany', 'argentina', 'colombia', 'belgium', 'portugal', 'cristianoronaldo', 'spain', 'mexico', 'tv', 'stadium', 'footballgame', 'brasil', 'worldcupqatar2022'], ['0z✨🚀', 'public', 'sale', 'is', 'live', '✨🚀n✅', 'polygon', 'contractn✅', 'verified', 'collectionn✅', 'pinkypunkssnn', 'eth', 'nft', 'nftcommmunity', 'nftlaunchnpinknmaticnpolygonnworldcupnbitkeepnbitrisenelementsnqatarworldcup2022nn', '159701304473474662420221127t234235000z✨🚀', 'sellling', 'out', 'very', 'fast', '✨🚀nsale', 'i

running iteration 2
running iteration 3
converged
1:  9 tweets
2:  20 tweets
3:  83 tweets
4:  14 tweets
5:  21 tweets
6:  13 tweets
7:  32 tweets
--> SSE : 10.267797392836691
[['enjoyed', 'this', 'guys', 'keep', 'smashing', 'it', '⚽👊🍻eng', 'england', 'worldcup', 'wba'], ['0z✨🚀', 'public', 'sale', 'is', 'live', '✨🚀n✅', 'polygon', 'contractn✅', 'verified', 'collectionn✅', 'pinkypunkssnn', 'eth', 'nft', 'nftcommmunity', 'nftlaunchnpinknmaticnpolygonnworldcupnbitkeepnbitrisenelementsnqatarworldcup2022nn', '159701304473474662420221127t234235000z✨🚀', 'sellling', 'out', 'very', 'fast', '✨🚀nsale', 'is', 'live', 'for', 'alln✅', 'polygon', 'contractn✅', 'verified', 'collectionn✅', 'pinky', 'punksn', 'eth', 'nft', 'nftcommmunity', 'nftlaunchnluckynmaticnnftswapnshibanbitkeepnqatarworldcup2022nworldcupnsolananbinance'], ['000zmoroccan', 'football', 'fans', 'raise', 'a', 'big', 'palestinian', 'flag', 'during', 'the', 'morocco', 'vs', 'belgium', 'match', 'at', 'the', 'fifaworldcup2022', 'in', 'qata

In [13]:
print(min_sse, math.sqrt(min_sse))
print(req_k)
print(req_cluster)

105.42766330034395 10.267797392836691
7
{2: [[['000zedgar', 'worldcup', 'driscoll', '世界杯', 'quenna', '偏门', 'iver', '卡塔尔世界杯', '159477859007977062420221127t235854000zksi', 'when', 'most', 'of', 'americans', 'recognized', 'it', 'as', 'soccer', 'worldcup', '159701713282659942420221127t235850000zrt', 'confident', 'and', 'ready', 'for', 'brasil', 'test', 'worldcup', 'swiss🇨🇭', '159542635099898266320221127t235825000zcontrol2xy', 'mexc', 'twitter', 'space', 'worldcup2022', 'special', 'episode', '1', 'is', 'here', 'lets', 'talk', 'about', 'the', 'worldcup', 'we', 'watched', 'in', 'the', 'past', 'years', 'set', 'a', 'reminder', 'join', 'the', 'vote', 'amp', 'earn', 'event', '159701691702950297620221127t235758000zrt', 'the', 'most', 'arabafricanmuslim', 'photo', 'from', 'the', 'worldcup', '159701690612465664020221127t235756000zrt', 'representing', 'united', 'in', 'the', 'first', 'week', 'of', 'the', 'worldcup', '😍nnmufc', '159701687434857676820221127t235748000zrt', 'moroccan', '🇲🇦', 'players', 'p

In [14]:
print(type(req_cluster))
#         print(clusters.keys())
#         print((max(len(clusters.values()))))
#         print(len(max(clusters.items(), key=lambda x:len(x[1]))[1]), max(clusters.items(), key=lambda x:len(x[1]))[0])
i = max(req_cluster.items(), key=lambda x:len(x[1]))[0]
print(centroids[i])

<class 'dict'>
['000zmoroccan', 'football', 'fans', 'raise', 'a', 'big', 'palestinian', 'flag', 'during', 'the', 'morocco', 'vs', 'belgium', 'match', 'at', 'the', 'fifaworldcup2022', 'in', 'qatar', 'nnworldcup2022', 'fifaworldcup', 'worldcup', 'fifaworldcupqatar2022', 'المغرب', 'المغرببلجيكا', '159701304119096115220221127t234234000zwatch', 'during', 'the', 'celebrations', 'over', 'their', 'countrys', '20', 'win', 'over', 'belgium', 'at', 'todays', 'fifaworldcup', 'match', 'moroccan', 'football', 'fans', 'chant', 'for', 'palestine', 'and', 'raise', 'palestinian', 'flags', 'in', 'qatar', 'todaynnworldcup2022', 'worldcup', 'fifaworldcupqatar2022', 'المغرب', 'المغرببلجيكا']


In [15]:
i = max(req_cluster.items(), key=lambda x:len(x[1]))[0]
print(centroids[i])

['000zmoroccan', 'football', 'fans', 'raise', 'a', 'big', 'palestinian', 'flag', 'during', 'the', 'morocco', 'vs', 'belgium', 'match', 'at', 'the', 'fifaworldcup2022', 'in', 'qatar', 'nnworldcup2022', 'fifaworldcup', 'worldcup', 'fifaworldcupqatar2022', 'المغرب', 'المغرببلجيكا', '159701304119096115220221127t234234000zwatch', 'during', 'the', 'celebrations', 'over', 'their', 'countrys', '20', 'win', 'over', 'belgium', 'at', 'todays', 'fifaworldcup', 'match', 'moroccan', 'football', 'fans', 'chant', 'for', 'palestine', 'and', 'raise', 'palestinian', 'flags', 'in', 'qatar', 'todaynnworldcup2022', 'worldcup', 'fifaworldcupqatar2022', 'المغرب', 'المغرببلجيكا']
